In [1]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from scipy import stats
from statsmodels.tsa.vector_ar.vecm import coint_johansen

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df_eurchf = pd.read_csv('/content/EUR-CHF Exchange Rate.csv')
df_trends = pd.read_excel('/content/World_GT_Full.xlsx')

In [3]:
df_trends['Date'] = pd.to_datetime(df_trends.iloc[:, 0], infer_datetime_format=True)
df_eurchf['Date'] = pd.to_datetime(df_eurchf.iloc[:, 0], infer_datetime_format=True)

In [4]:
df_trends.iloc[:,1:] = df_trends.iloc[:,1:].diff()
df_trends

,Date,swiss franc,Europe,EUR,Switzerland,CHF,NASDAQ,S&P500,VIX,volatility,Germany,UK,sterling pound,GBP,Greece,Italy,Spain,France,DAX,unemployment,recession,euro,crisis,fed,finance,financial crisis,federal reserve,gold,inflation,treasuries,OPEC,contagion,corruption,disaster,futures,forwards,president,parliament,aid,EU,...,steel,silver,bronze,corn,commodities,natural gas,agriculture,emerging market,trade,interest rate,GDP,exports,imports,jobs,tax,tariff,uncertainty,gini,inequality,insurance,pension,liquidity,diversification,alpha,currency,CAPM,fixed income,commission,SEC,yield,real estate,eurodollar,regulations,deficit,yield curve,LIBOR,2008,bubble,energy,1939
0,2004-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-01-02,0.0,16.0,9.0,2.0,-1.0,10.0,-27.0,17.0,31.0,1.0,-2.0,0.0,1.0,-4.0,-1.0,-2.0,-6.0,2.0,2.0,0.0,-1.0,-6.0,7.0,20.0,0.0,0.0,1.0,19.0,0.0,8.0,0.0,-58.0,-1.0,3.0,-24.0,0.0,-3.0,10.0,0.0,...,-2.0,-13.0,-34.0,-13.0,18.0,19.0,6.0,0.0,-5.0,14.0,-5.0,25.0,11.0,2.0,5.0,34.0,15.0,-11.0,0.0,24.0,-13.0,0.0,-8.0,6.0,6.0,0.0,0.0,5.0,-3.0,35.0,10.0,0.0,-4.0,8.0,0.0,8.0,0.0,-16.0,6.0,10.0
2,2004-01-03,0.0,0.0,-10.0,-3.0,0.0,-6.0,0.0,-19.0,-31.0,-1.0,-3.0,0.0,-1.0,-1.0,0.0,-3.0,4.0,-2.0,0.0,0.0,0.0,5.0,-4.0,-16.0,0.0,10.0,-1.0,-6.0,0.0,-6.0,0.0,2.0,10.0,-7.0,30.0,0.0,0.0,-6.0,0.0,...,6.0,8.0,4.0,-1.0,-21.0,2.0,1.0,0.0,-4.0,-16.0,25.0,12.0,-21.0,-1.0,-6.0,-17.0,-3.0,0.0,0.0,-16.0,2.0,0.0,-8.0,-2.0,0.0,0.0,17.0,-5.0,8.0,-25.0,-2.0,0.0,8.0,7.0,0.0,-14.0,0.0,13.0,7.0,32.0
3,2004-01-04,19.0,-12.0,2.0,7.0,1.0,3.0,22.0,12.0,0.0,1.0,0.0,26.0,1.0,2.0,0.0,11.0,-5.0,1.0,1.0,2.0,0.0,0.0,-3.0,-3.0,0.0,-19.0,1.0,-8.0,0.0,2.0,0.0,7.0,-5.0,5.0,-38.0,0.0,0.0,-1.0,1.0,...,-5.0,-6.0,13.0,2.0,8.0,-16.0,3.0,14.0,5.0,7.0,3.0,-36.0,-25.0,2.0,1.0,23.0,8.0,5.0,10.0,-6.0,8.0,27.0,0.0,-5.0,7.0,0.0,-17.0,4.0,-2.0,1.0,5.0,0.0,-14.0,-9.0,0.0,7.0,0.0,18.0,5.0,-39.0
4,2004-01-05,-19.0,-7.0,-4.0,-8.0,-3.0,-5.0,-22.0,18.0,0.0,-3.0,-50.0,-26.0,-1.0,-2.0,1.0,-14.0,-16.0,-2.0,0.0,-2.0,-2.0,9.0,1.0,13.0,0.0,-3.0,9.0,2.0,0.0,-4.0,0.0,-15.0,5.0,-7.0,0.0,2.0,-1.0,-6.0,-2.0,...,-4.0,4.0,2.0,-6.0,1.0,17.0,25.0,-14.0,1.0,28.0,51.0,5.0,-16.0,4.0,5.0,-17.0,-20.0,9.0,49.0,4.0,-37.0,-27.0,0.0,11.0,11.0,0.0,0.0,9.0,35.0,13.0,11.0,0.0,7.0,38.0,0.0,-16.0,0.0,-38.0,-3.0,-6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6465,2021-09-13,-1.0,0.0,4.0,0.0,6.0,32.0,17.0,8.0,3.0,1.0,-2.0,1.0,1.0,-1.0,0.0,0.0,1.0,13.0,2.0,0.0,1.0,2.0,7.0,35.0,0.0,4.0,-1.0,7.0,2.0,-1.0,-1.0,3.0,2.0,10.0,0.0,0.0,3.0,5.0,0.0,...,5.0,0.0,-1.0,-6.0,21.0,12.0,2.0,0.0,-7.0,3.0,5.0,9.0,1.0,2.0,8.0,8.0,2.0,2.0,5.0,15.0,17.0,2.0,-1.0,2.0,2.0,2.0,1.0,2.0,-1.0,10.0,1.0,0.0,6.0,1.0,0.0,2.0,0.0,-8.0,22.0,-2.0
6466,2021-09-14,1.0,0.0,0.0,0.0,-1.0,2.0,0.0,-2.0,-1.0,0.0,1.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,-1.0,-4.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.0,0.0,6.0,1.0,1.0,0.0,2.0,2.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,...,-1.0,-1.0,1.0,0.0,30.0,5.0,1.0,2.0,-3.0,2.0,-1.0,-7.0,1.0,0.0,3.0,-1.0,-1.0,0.0,0.0,1.0,-2.0,1.0,1.0,-1.0,2.0,2.0,0.0,1.0,0.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,4.0,-1.0
6467,2021-09-15,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,-7.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,3.0,1.0,-1.0,0.0,-12.0,0.0,0.0,-1.0,1.0,-1.0,4.0,11.0,-1.0,0.0,0.0,7.0,4.0,1.0,1.0,3.0,-2.0,0.0,-1.0,2.0,7.0,1.0,2.0,0.0,0.0,-4.0,0.0,0.0,0.0,-7.0,-1.0,0.0,0.0,0.0,2.0,3.0
6468,2021-09-16,0.0,0.0,0.0,-1.0,-2.0,-5.0,-4.0,-1.0,-1.0,-

In [5]:
ts_df = df_eurchf.merge(df_trends, how = 'inner', on = ['Date'])

In [6]:
ts_df = ts_df.set_index(ts_df.iloc[:,0])
ts_df = ts_df.drop(['Date', ' value'], axis = 1)
ts_df = ts_df.iloc[1:,:]
ts_df

,log_difference,swiss franc,Europe,EUR,Switzerland,CHF,NASDAQ,S&P500,VIX,volatility,Germany,UK,sterling pound,GBP,Greece,Italy,Spain,France,DAX,unemployment,recession,euro,crisis,fed,finance,financial crisis,federal reserve,gold,inflation,treasuries,OPEC,contagion,corruption,disaster,futures,forwards,president,parliament,aid,EU,...,steel,silver,bronze,corn,commodities,natural gas,agriculture,emerging market,trade,interest rate,GDP,exports,imports,jobs,tax,tariff,uncertainty,gini,inequality,insurance,pension,liquidity,diversification,alpha,currency,CAPM,fixed income,commission,SEC,yield,real estate,eurodollar,regulations,deficit,yield curve,LIBOR,2008,bubble,energy,1939
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-01-05,0.002498,-19.0,-7.0,-4.0,-8.0,-3.0,-5.0,-22.0,18.0,0.0,-3.0,-50.0,-26.0,-1.0,-2.0,1.0,-14.0,-16.0,-2.0,0.0,-2.0,-2.0,9.0,1.0,13.0,0.0,-3.0,9.0,2.0,0.0,-4.0,0.0,-15.0,5.0,-7.0,0.0,2.0,-1.0,-6.0,-2.0,...,-4.0,4.0,2.0,-6.0,1.0,17.0,25.0,-14.0,1.0,28.0,51.0,5.0,-16.0,4.0,5.0,-17.0,-20.0,9.0,49.0,4.0,-37.0,-27.0,0.0,11.0,11.0,0.0,0.0,9.0,35.0,13.0,11.0,0.0,7.0,38.0,0.0,-16.0,0.0,-38.0,-3.0,-6.0
2004-01-06,0.002875,0.0,34.0,5.0,2.0,2.0,21.0,0.0,-20.0,30.0,5.0,77.0,0.0,0.0,9.0,0.0,7.0,25.0,2.0,3.0,0.0,3.0,-6.0,8.0,46.0,0.0,27.0,-8.0,44.0,0.0,3.0,0.0,41.0,-7.0,16.0,0.0,0.0,8.0,10.0,3.0,...,14.0,-5.0,4.0,16.0,11.0,-11.0,-14.0,12.0,20.0,-18.0,-69.0,22.0,13.0,-2.0,2.0,36.0,10.0,-9.0,-42.0,50.0,48.0,15.0,11.0,2.0,8.0,22.0,39.0,5.0,2.0,1.0,-28.0,15.0,46.0,-4.0,74.0,75.0,0.0,6.0,32.0,30.0
2004-01-07,-0.000064,0.0,-13.0,-3.0,1.0,2.0,-3.0,0.0,-10.0,2.0,-2.0,-21.0,0.0,0.0,-1.0,1.0,7.0,-9.0,-1.0,-2.0,1.0,0.0,6.0,14.0,-16.0,0.0,1.0,1.0,-23.0,0.0,3.0,0.0,-32.0,3.0,-10.0,8.0,0.0,-4.0,-2.0,-1.0,...,-2.0,2.0,-10.0,-9.0,-7.0,29.0,18.0,-3.0,2.0,-2.0,18.0,-2.0,9.0,0.0,2.0,-25.0,17.0,-1.0,2.0,-24.0,-33.0,14.0,-11.0,-9.0,1.0,-12.0,19.0,-2.0,-24.0,-8.0,6.0,-4.0,-22.0,-10.0,-34.0,-50.0,0.0,2.0,3.0,-25.0
2004-01-08,-0.001341,0.0,7.0,5.0,1.0,-2.0,16.0,0.0,6.0,-21.0,-1.0,3.0,8.0,0.0,-5.0,0.0,3.0,4.0,3.0,-3.0,0.0,-1.0,-7.0,-8.0,-8.0,2.0,-5.0,-2.0,0.0,0.0,-2.0,0.0,-5.0,0.0,12.0,15.0,0.0,2.0,3.0,1.0,...,2.0,-4.0,5.0,3.0,5.0,-8.0,-11.0,-9.0,8.0,6.0,-8.0,9.0,18.0,-2.0,-3.0,16.0,3.0,2.0,25.0,5.0,18.0,-11.0,0.0,-5.0,-1.0,-10.0,-20.0,-3.0,-13.0,5.0,-5.0,-11.0,21.0,18.0,-40.0,26.0,0.0,-1.0,-7.0,-3.0
2004-01-09,0.000958,0.0,8.0,-1.0,1.0,0.0,-17.0,0.0,-4.0,0.0,1.0,-6.0,-8.0,0.0,-1.0,-1.0,-7.0,-1.0,-1.0,1.0,0.0,1.0,-3.0,4.0,7.0,2.0,22.0,-1.0,-10.0,50.0,3.0,1.0,3.0,1.0,-1.0,-8.0,2.0,-2.0,-3.0,-1.0,...,-1.0,3.0,39.0,0.0,-9.0,-13.0,10.0,0.0,-15.0,10.0,25.0,-4.0,-3.0,-2.0,0.0,-8.0,-30.0,1.0,-24.0,-10.0,1.0,-6.0,9.0,0.0,-14.0,0.0,-28.0,-2.0,19.0,-12.0,7.0,0.0,-3.0,-13.0,0.0,-24.0,0.0,-4.0,-13.0,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-09,-0.005245,-1.0,-3.0,0.0,-2.0,-1.0,0.0,8.0,-2.0,0.0,0.0,0.0,-1.0,0.0,-2.0,0.0,-1.0,-3.0,3.0,-3.0,0.0,0.0,1.0,2.0,-4.0,2.0,0.0,-2.0,-4.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,-1.0,1.0,0.0,...,-3.0,-1.0,3.0,-1.0,-13.0,-4.0,2.0,1.0,1.0,-1.0,0.0,3.0,2.0,0.0,-1.0,0.0,-4.0,-1.0,1.0,-2.0,-7.0,-3.0,0.0,-1.0,0.0,5.0,-1.0,0.0,0.0,-1.0,-1.0,1.0,0.0,17.0,1.0,-1.0,0.0,1.0,-4.0,-1.0
2021-09-10,0.000369,0.0,-3.0,-1.0,-1.0,-1.0,-4.0,-4.0,-6.0,-2.0,-1.0,-1.0,-1.0,-1.0,-2.0,0.0,0.0,-1.0,-4.0,-2.0,-1.0,0.0,-2.0,-1.0,-6.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,-1.0,-3.0,-1.0,0.0,0.0,-2.0,0.0,...,-2.0,0.0,0.0,1.0,7.0,-4.0,-3.0,1.0,2.0,-2.0,0.0,-2.0,-2.0,-1.0,-2.0,-3.0,1.0,-1.0,-2.0,-3.0,-4.0,-1.0,0.0,-2.0,-2.0,-3.0,0.0,-1.0,-3.0,-4.0,-1.0,-1.0,-4.0,-10.0,-2.0,0.0,0.0,2.0,-6.0,-1.0
2021-09-11,0.000369,1.0,-3.0,-3.0,0.0,-3.0,-29.0,-5.0,-3.0,-2.0,-1.0,0.0,1.0,-1.0,0.0,0.0,-1.0,0.0,-11.0,-5.0,0.0,-1.0,-2.0,-6.0,-22.0,1.0,-3

In [7]:
#Convert EUR/CHF to CHF/EUR Returns
ts_df['log_difference'] = ts_df['log_difference'] * -1

In [43]:
def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,0,13)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    stats_res = []

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)
        result_list = [adjust(round(trace,2), 9), adjust(cvt, 8)]
        stats_res.append(result_list)

    return(stats_res)

In [45]:
i=1
subset_ts_df = pd.concat([ts_df['log_difference'], ts_df.iloc[:,i:i+10]], axis=1)
cointegration_test(subset_ts_df, 0.05)

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
log_difference ::  4042.45   > 285.1402  =>   True
swiss franc ::  3403.89   > 239.2468  =>   True
Europe ::  2877.24   > 197.3772  =>   True
EUR    ::  2445.76   > 159.529   =>   True
Switzerland ::  2027.71   > 125.6185  =>   True
CHF    ::  1623.02   > 95.7542   =>   True
NASDAQ ::  1280.53   > 69.8189   =>   True
S&P500 ::  962.61    > 47.8545   =>   True
VIX    ::  660.61    > 29.7961   =>   True
volatility ::  415.04    > 15.4943   =>   True
Germany ::  192.39    > 3.8415    =>   True


In [52]:
coint = []
testing = []

In [53]:
for i in range(1, ts_df.shape[1], 10):

  subset_ts_df = pd.concat([ts_df['log_difference'], ts_df.iloc[:,i:i+10]], axis=1)
  coint = cointegration_test(subset_ts_df, 0.05)
  testing.append(coint)

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
log_difference ::  4042.45   > 285.1402  =>   True
swiss franc ::  3403.89   > 239.2468  =>   True
Europe ::  2877.24   > 197.3772  =>   True
EUR    ::  2445.76   > 159.529   =>   True
Switzerland ::  2027.71   > 125.6185  =>   True
CHF    ::  1623.02   > 95.7542   =>   True
NASDAQ ::  1280.53   > 69.8189   =>   True
S&P500 ::  962.61    > 47.8545   =>   True
VIX    ::  660.61    > 29.7961   =>   True
volatility ::  415.04    > 15.4943   =>   True
Germany ::  192.39    > 3.8415    =>   True
Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
log_difference ::  3515.37   > 285.1402  =>   True
UK     ::  2999.62   > 239.2468  =>   True
sterling pound ::  2547.14   > 197.3772  =>   True
GBP    ::  2130.97   > 159.529   =>   True
Greece ::  1721.83   > 125.6185  =>   True
Italy  ::  1366.62   > 95.7542   =>   True
Spain  ::  1038.3    > 69.8189   =>   True
France ::

In [54]:
testing

[[['4042.45  ', '285.1402'],
  ['3403.89  ', '239.2468'],
  ['2877.24  ', '197.3772'],
  ['2445.76  ', '159.529 '],
  ['2027.71  ', '125.6185'],
  ['1623.02  ', '95.7542 '],
  ['1280.53  ', '69.8189 '],
  ['962.61   ', '47.8545 '],
  ['660.61   ', '29.7961 '],
  ['415.04   ', '15.4943 '],
  ['192.39   ', '3.8415  ']],
 [['3515.37  ', '285.1402'],
  ['2999.62  ', '239.2468'],
  ['2547.14  ', '197.3772'],
  ['2130.97  ', '159.529 '],
  ['1721.83  ', '125.6185'],
  ['1366.62  ', '95.7542 '],
  ['1038.3   ', '69.8189 '],
  ['730.43   ', '47.8545 '],
  ['437.22   ', '29.7961 '],
  ['204.52   ', '15.4943 '],
  ['34.0     ', '3.8415  ']],
 [['3559.68  ', '285.1402'],
  ['3090.95  ', '239.2468'],
  ['2633.4   ', '197.3772'],
  ['2220.05  ', '159.529 '],
  ['1846.72  ', '125.6185'],
  ['1500.25  ', '95.7542 '],
  ['1173.78  ', '69.8189 '],
  ['861.34   ', '47.8545 '],
  ['590.46   ', '29.7961 '],
  ['331.24   ', '15.4943 '],
  ['86.3     ', '3.8415  ']],
 [['4074.11  ', '285.1402'],
  ['3535.13

In [131]:
test_stat = []
crit_val_95 = []

In [132]:
for i in range(0, 15, 1):

  test_stat.append(testing[i][1][0])
  test_stat.append(testing[i][2][0])
  test_stat.append(testing[i][3][0])
  test_stat.append(testing[i][4][0])
  test_stat.append(testing[i][5][0])
  test_stat.append(testing[i][6][0])
  test_stat.append(testing[i][7][0])
  test_stat.append(testing[i][8][0])
  test_stat.append(testing[i][9][0])
  test_stat.append(testing[i][10][0])

  crit_val_95.append(testing[i][1][1])
  crit_val_95.append(testing[i][2][1])
  crit_val_95.append(testing[i][3][1])
  crit_val_95.append(testing[i][4][1])
  crit_val_95.append(testing[i][5][1])
  crit_val_95.append(testing[i][6][1])
  crit_val_95.append(testing[i][7][1])
  crit_val_95.append(testing[i][8][1])
  crit_val_95.append(testing[i][9][1])
  crit_val_95.append(testing[i][10][1])

In [133]:
test_stat.append(testing[15][1][0])
test_stat.append(testing[15][2][0])
test_stat.append(testing[15][3][0])
test_stat.append(testing[15][4][0])

crit_val_95.append(testing[15][1][1])
crit_val_95.append(testing[15][2][1])
crit_val_95.append(testing[15][3][1])
crit_val_95.append(testing[15][4][1])

In [134]:
col_titles = []

In [135]:
for i in range(1, ts_df.shape[1], 10):

  subset_ts_df = pd.concat([ts_df['log_difference'], ts_df.iloc[:,i:i+10]], axis=1)
  col_titles.append(subset_ts_df.columns)

In [136]:
col_titles

[Index(['log_difference', 'swiss franc', 'Europe', 'EUR', 'Switzerland', 'CHF',
        'NASDAQ', 'S&P500', 'VIX', 'volatility', 'Germany'],
       dtype='object'),
 Index(['log_difference', 'UK', 'sterling pound', 'GBP', 'Greece', 'Italy',
        'Spain', 'France', 'DAX', 'unemployment', 'recession'],
       dtype='object'),
 Index(['log_difference', 'euro', 'crisis', 'fed', 'finance',
        'financial crisis', 'federal reserve', 'gold', 'inflation',
        'treasuries', 'OPEC'],
       dtype='object'),
 Index(['log_difference', 'contagion', 'corruption', 'disaster', 'futures',
        'forwards', 'president', 'parliament', 'aid', 'EU', 'European Union'],
       dtype='object'),
 Index(['log_difference', 'stimulus', 'capital gains', 'climate change', 'OECD',
        'conflict', 'environment', 'ECB', 'SNB', 'bank', 'IMF'],
       dtype='object'),
 Index(['log_difference', 'USD', 'US Dollar', 'China', 'US', 'MSCI', 'home',
        'dividend', 'labor', 'buy', 'financial market'],
   

In [137]:
col_titles[0][1]

'swiss franc'

In [138]:
trend_titles = []

In [139]:
for i in range(0, 15, 1):

  trend_titles.append(col_titles[i][1])
  trend_titles.append(col_titles[i][2])
  trend_titles.append(col_titles[i][3])
  trend_titles.append(col_titles[i][4])
  trend_titles.append(col_titles[i][5])
  trend_titles.append(col_titles[i][6])
  trend_titles.append(col_titles[i][7])
  trend_titles.append(col_titles[i][8])
  trend_titles.append(col_titles[i][9])
  trend_titles.append(col_titles[i][10])

In [140]:
trend_titles.append(col_titles[15][1])
trend_titles.append(col_titles[15][2])
trend_titles.append(col_titles[15][3])
trend_titles.append(col_titles[15][4])

In [141]:
print(len(test_stat))
print(len(crit_val_95))
print(len(trend_titles))

154
154
154


In [151]:
tit = pd.DataFrame(trend_titles)
tes = pd.DataFrame(test_stat)
cri = pd.DataFrame(crit_val_95)

In [152]:
cointegration_test_table = pd.concat([tit, tes, cri], axis=1)

In [155]:
cointegration_test_table.columns = ['Trend Name', 'Test Statistic', '95% Critical Values']

In [156]:
cointegration_test_table

,Trend Name,Test Statistic,95% Critical Values
0,swiss franc,3403.89,239.2468
1,Europe,2877.24,197.3772
2,EUR,2445.76,159.529
3,Switzerland,2027.71,125.6185
4,CHF,1623.02,95.7542
...,...,...,...
149,LIBOR,58.13,3.8415
150,2008,1081.32,47.8545
151,bubble,731.75,29.7961
152,energy,403.59,15.4943


In [157]:
with open('CHFEUR_CointegrationTable.tex', 'w') as tf:
     tf.write(cointegration_test_table.to_latex())